In [1]:
import pandas as pd
import numpy as np
import math
import glob
import os

In [2]:
parent_folders = ['/home/bvelasco/CovidVax_DM/Results/binary_eof/grace_period_180/*/*.csv',
                 '/home/bvelasco/CovidVax_DM/Results/binary_eof/grace_period_360/*/*.csv',
                 '/home/bvelasco/CovidVax_DM/Results/survival/grace_period_180/*/*.csv',
                 '/home/bvelasco/CovidVax_DM/Results/survival/grace_period_360/*/*.csv']

In [10]:
data=None

for _, folder in enumerate(parent_folders):
    
    result_files = glob.glob(folder)
    
    for index, file  in enumerate(result_files):

        if index==0:
            data = pd.read_csv(file)
            data.drop(['Unnamed: 0'], inplace=True, axis=1)
        else:
            newdata = pd.read_csv(file)
            newdata.drop(['Unnamed: 0', 'Intervention'], inplace=True, axis=1)
            data = pd.concat([data, newdata], axis=1)
        
    if  'binary_eof' in folder:
        result_data = pd.concat([data['g-form mean (NICE)'].mean(axis=1),
        data['g-form mean (NICE)'].mean(axis=1)-1.96*data['g-form mean (NICE)'].std(axis=1), 
        data['g-form mean (NICE)'].mean(axis=1)+1.96*data['g-form mean (NICE)'].std(axis=1),
        data['Mean ratio'].mean(axis=1),
        data['Mean ratio'].mean(axis=1)-1.96*data['Mean ratio'].std(axis=1),
        data['Mean ratio'].mean(axis=1)+1.96*data['Mean ratio'].std(axis=1)], axis=1)

        result_data.columns = [
            'g-form mean (NICE)', 'g-form mean (NICE) lower', 'g-form mean (NICE) upper',
            'Mean ratio', 'Mean ratio lower', 'Mean ratio upper'
        ]

        result_data['Probability [95% CI]'] = result_data.apply(
            lambda row: f"{row['g-form mean (NICE)']:.3f} [{row['g-form mean (NICE) lower']:.3f}, {row['g-form mean (NICE) upper']:.3f}]",
            axis=1
        )

        result_data['Probability ratio [95% CI]'] = result_data.apply(
            lambda row: f"{row['Mean ratio']:.3f} [{row['Mean ratio lower']:.3f}, {row['Mean ratio upper']:.3f}]",
            axis=1
        )
        
        result_data = result_data.set_axis(data['Intervention'], axis=0)
        print(result_data[['Probability [95% CI]', 'Probability ratio [95% CI]']].to_latex(bold_rows=True, escape=True))

    elif 'survival' in folder:
        result_data = pd.concat([
        data['time'].mean(axis=1).astype(int),
        data['g-form risk (NICE-parametric)'].mean(axis=1),
        data['g-form risk (NICE-parametric)'].mean(axis=1)-1.96*data['g-form risk (NICE-parametric)'].std(axis=1),
        data['g-form risk (NICE-parametric)'].mean(axis=1)+1.96*data['g-form risk (NICE-parametric)'].std(axis=1),
        data['Risk Ratio'].mean(axis=1),
        data['Risk Ratio'].mean(axis=1)-1.96*data['Risk Ratio'].std(axis=1),
        data['Risk Ratio'].mean(axis=1)+1.96*data['Risk Ratio'].std(axis=1)], axis=1)

        result_data.columns = [
            'time', 'g-form risk (NICE-parametric)', 'g-form risk (NICE-parametric) lower', 'g-form risk (NICE-parametric) upper',
            'Risk Ratio', 'Risk Ratio lower', 'Risk Ratio upper'
        ]

        result_data['Survival [95% CI]'] = result_data.apply(
            lambda row: f"{row['g-form risk (NICE-parametric)']:.3f} [{row['g-form risk (NICE-parametric) lower']:.3f}, {row['g-form risk (NICE-parametric) upper']:.3f}]",
            axis=1
        )

        result_data['Survival ratio [95% CI]'] = result_data.apply(
            lambda row: f"{row['Risk Ratio']:.3f} [{row['Risk Ratio lower']:.3f}, {row['Risk Ratio upper']:.3f}]",
            axis=1
        )
        
        result_data = result_data.set_axis(data['Intervention'], axis=0)
        print(result_data[['time', 'Survival [95% CI]', 'Survival ratio [95% CI]']].to_latex(bold_rows=True, escape=True))
    

\begin{tabular}{lll}
\toprule
 & Probability [95\% CI] & Probability ratio [95\% CI] \\
Intervention &  &  \\
\midrule
\textbf{Natural course} & 0.092 [0.083, 0.100] & 1.000 [1.000, 1.000] \\
\textbf{Never treat} & 0.121 [0.114, 0.129] & 1.325 [1.248, 1.402] \\
\textbf{Treat on Vacuna only at t1} & 0.088 [0.079, 0.097] & 0.962 [0.949, 0.975] \\
\textbf{Treat on Vacuna only at t1 \& t2} & 0.087 [0.078, 0.095] & 0.943 [0.929, 0.957] \\
\textbf{Treat on Vacuna at t1, t2 \& t3} & 0.088 [0.078, 0.098] & 0.959 [0.909, 1.009] \\
\bottomrule
\end{tabular}

\begin{tabular}{lll}
\toprule
 & Probability [95\% CI] & Probability ratio [95\% CI] \\
Intervention &  &  \\
\midrule
\textbf{Natural course} & 0.032 [-0.004, 0.069] & 1.000 [1.000, 1.000] \\
\textbf{Never treat} & 0.046 [-0.008, 0.101] & 1.413 [1.220, 1.605] \\
\textbf{Treat on Vacuna only at t1} & 0.031 [-0.004, 0.067] & 0.972 [0.954, 0.990] \\
\textbf{Treat on Vacuna only at t1 \& t2} & 0.031 [-0.004, 0.065] & 0.946 [0.917, 0.976] \\
\te